In [5]:
import webber
from webber.xcoms import Promise

It's useful to recall that Webber DAGs can represent any Python callable, allowing orchestrators a great deal of flexibility in constructing nodes, whether using function callables, lambda functions, or class initalizations.

In [6]:
dag = webber.DAG()
n1 = dag.add_node(str, "Hello World!")
n2 = dag.add_node(print, Promise(n1))
_ = dag.add_edge(n1, n2)
dag.execute()

2024-06-23 11:59:56,260           print: Hello World!


Another example leveraging Promises as inputs to Lambda functions...

In [7]:
dag = webber.DAG()
n1 = dag.add_node(str, "Hello World!")
n2 = dag.add_node(lambda n: print(n), Promise(n1))
_ = dag.add_edge(n1, n2)
dag.execute()

2024-06-23 11:59:57,642        <lambda>: Hello World!


Nice, perhaps we can make this a bit more complicated...

In [8]:
from random import randint
from time import time

dag = webber.DAG()
n1 = dag.add_node(randint, 1, 99)
n2 = dag.add_node(randint, 1, 10)
n3 = dag.add_node(lambda i, j: print(f"{i} mod {j} = {i % j}"), Promise(n1), Promise(n2))
dag.add_edge(n1, n3)
dag.add_edge(n2, n3)

t = time()
dag.execute()
print(f"Execution time:", time() - t)

2024-06-23 12:00:02,980        <lambda>: 36 mod 7 = 1
Execution time: 0.23811697959899902


Of course, Webber's performance here is great in comparison to Dagster and Airflow, but sequential execution will still be magnitudes faster for the example above, if not practically zero.

In [9]:
t = time()
i, j = randint(1, 99), randint(1, 10)
print(f"{i} mod {j} = {i % j}")
print(f"Execution time:", time() - t)

90 mod 3 = 0
Execution time: 0.00099945068359375


Given this new abstraction, we should be using Webber to handle longer running processes in parallel.
Regardless, experimentation like this is cool - and starting here is *critical* to abstracting the way we think about processes and their dependencies.